In [ ]:
import json
from collections import defaultdict
from datetime import datetime, timedelta

PRICE_LIMIT = 200
START_WEEKDAYS = {4, 5}  # Friday, Saturday
END_WEEKDAYS = {5, 6, 0}  # Saturday, Sunday, Monday

def load_data(file_path: str) -> dict:
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

def parse_dates(data: list) -> None:
    for item in data:
        item['date'] = datetime.strptime(item['date'], '%Y-%m-%d')

def filter_by_weekdays(data: list, weekdays: set) -> list:
    return [item for item in data if item['date'].weekday() in weekdays]

def filter_by_price(data: list, price_limit: int) -> list:
    return [item for item in data if item['price'] < price_limit]

def get_week_number(date: datetime) -> int:
    if date.weekday() == 0:
        date -= timedelta(days=1)
    return date.isocalendar()[1]

def add_week_number(data: list) -> None:
    for item in data:
        item['week'] = get_week_number(item['date'])

def group_flights_by_key(data: list, key: str) -> dict:
    grouped = defaultdict(list)
    for item in data:
        grouped[item[key]].append(item)
    return dict(grouped)

def filter_cheapest_weekend_flights(data: dict) -> dict:
    result = {}
    for key, flights in data.items():
        weekly_flights = defaultdict(list)
        for flight in flights:
            weekly_flights[flight['week']].append(flight)
        
        result[key] = [min(week, key=lambda x: x['price']) for week in weekly_flights.values()]
    return result

def find_available_trips(wroclaw_to_anywhere: dict, anywhere_to_wroclaw: dict) -> dict:
    possible_iatas = set(wroclaw_to_anywhere).intersection(anywhere_to_wroclaw)
    available_trips = defaultdict(list)
    for iata in possible_iatas:
        start_flights = wroclaw_to_anywhere[iata]
        back_flights = anywhere_to_wroclaw[iata]
        available_weekends = set(flight['week'] for flight in start_flights).intersection(flight['week'] for flight in back_flights)
        
        start_flights = [flight for flight in start_flights if flight['week'] in available_weekends]
        back_flights = [flight for flight in back_flights if flight['week'] in available_weekends]
        
        for weekend in available_weekends:
            start_flight = next(flight for flight in start_flights if flight['week'] == weekend)
            back_flight = next(flight for flight in back_flights if flight['week'] == weekend)
            available_trips[iata].append({
                'start_flight': start_flight,
                'back_flight': back_flight,
                'total_price': start_flight['price'] + back_flight['price']
            })
    return available_trips

def find_cheapest_trips(trips: dict) -> dict:
    return {iata: min(flights, key=lambda x: x['total_price']) for iata, flights in trips.items()}

def print_flights_sorted_by_price(trips: dict) -> None:
    sorted_flights = sorted(trips.items(), key=lambda x: x[1]['total_price'])
    for key, flight_info in sorted_flights:
        start_flight = flight_info['start_flight']
        back_flight = flight_info['back_flight']
        total_price = flight_info['total_price']
        
        start_date_str = start_flight['date'].strftime("%Y-%m-%d (%A)")
        back_date_str = back_flight['date'].strftime("%Y-%m-%d (%A)")
        
        print(f"{start_flight['end_name']}, Start Date: {start_date_str}, Return Date: {back_date_str}, Total Price: {total_price}")
        print("-" * 40)

def main():
    data = load_data('date_price_list.json')
    
    wroclaw_to_anywhere = data['wrocław_to_anywhere']
    anywhere_to_wroclaw = data['anywhere_to_wrocław']
    
    parse_dates(wroclaw_to_anywhere)
    parse_dates(anywhere_to_wroclaw)
    
    wroclaw_to_anywhere_filtered = filter_by_weekdays(wroclaw_to_anywhere, START_WEEKDAYS)
    anywhere_to_wroclaw_filtered = filter_by_weekdays(anywhere_to_wroclaw, END_WEEKDAYS)
    
    wroclaw_to_anywhere_filtered = filter_by_price(wroclaw_to_anywhere_filtered, PRICE_LIMIT)
    anywhere_to_wroclaw_filtered = filter_by_price(anywhere_to_wroclaw_filtered, PRICE_LIMIT)
    
    add_week_number(wroclaw_to_anywhere_filtered)
    add_week_number(anywhere_to_wroclaw_filtered)
    
    grouped_wroclaw_to_anywhere = group_flights_by_key(wroclaw_to_anywhere_filtered, 'end')
    grouped_anywhere_to_wroclaw = group_flights_by_key(anywhere_to_wroclaw_filtered, 'start')
    
    grouped_wroclaw_to_anywhere = filter_cheapest_weekend_flights(grouped_wroclaw_to_anywhere)
    grouped_anywhere_to_wroclaw = filter_cheapest_weekend_flights(grouped_anywhere_to_wroclaw)
    
    available_trips = find_available_trips(grouped_wroclaw_to_anywhere, grouped_anywhere_to_wroclaw)
    cheapest_trips = find_cheapest_trips(available_trips)
    
    print_flights_sorted_by_price(cheapest_trips)

if __name__ == "__main__":
    main()